In [ ]:
import pandas as pd
import numpy as np

### Import Dataframes

In [ ]:
#cnumber to clinic subset info
cnum_clinic = pd.read_csv(#redacted)

#baseline isos
baseline_isos = pd.read_csv(#redacted)

#### Contruct In Vivo Dataframe

In [ ]:
print(cnum_clinic.visit_raw.unique())

In [ ]:
cnum_clinic.subject_id.unique()

In [ ]:
#Get CFU/mL at 0, 4 ,24
days1and2 = cnum_clinic[(cnum_clinic["visit_raw"] == "Screening") | (cnum_clinic["visit_raw"] == "V2/D2")]

#Noting all points where there is a day 2 post 4h points
day2_postDose = days1and2[(days1and2.visit_raw == "V2/D2") & (days1and2.specimen_collection_time_description == "4 hr Post Dose")]
#print(day2_postDose.subject_id.value_counts())

#Noting patients whose third point is post dosing on day 2
third_point_post_dose = day2_postDose[~day2_postDose.subject_id.isin(four_points_list)]
third_point_post_dose_list = list(third_point_post_dose.subject_id)
#print(third_point_post_dose_list)

#drop the day 2 post-dose points
indices_to_drop = days1and2[(days1and2.visit_raw == "V2/D2") & (days1and2.specimen_collection_time_description == "4 hr Post Dose")].index
days1and2_0424 = days1and2.drop(indices_to_drop)

#Need to concat the "speciment_collection_time_description" and "visit_raw" to get the three time descriptors

#Start with changing "Pre First Dose" to "Pre-Dose"
pre_dose = lambda x: "Pre-Dose" if x == "Pre First Dose" else x
days1and2_0424.specimen_collection_time_description = days1and2_0424.specimen_collection_time_description.apply(pre_dose)

#Concat the columns
days1and2_0424["Timepoint"] = days1and2_0424.visit_raw.str.cat(days1and2_0424.specimen_collection_time_description, sep="_")

#Pivot to get into patient, 0h, 4h, 24h columns

#there are two duplicate isolates in this dataframe. Need to group and average CFU_ml
duplicates = days1and2_0424.duplicated(subset=['subject_id', 'Timepoint'], keep=False)
# print(days1and2_0424[duplicates])

in_vivo_grouped = days1and2_0424.groupby(["subject_id", "Timepoint"])["CFU_ml"].max().reset_index()

in_vivo_pivot = in_vivo_grouped.pivot(
    columns = "Timepoint",
    index = "subject_id",
    values = "CFU_ml").reset_index()

in_vivo_pivot = in_vivo_pivot[["subject_id", "Screening_Pre-Dose", "Screening_4 hr Post Dose", "V2/D2_Pre-Dose"]]
in_vivo_pivot.iloc[:, 1:] = in_vivo_pivot.iloc[:, 1:].apply(np.log10)
in_vivo_pivot.columns= ["Patient", "Patient_0h_logCFU", "Patient_4h_logCFU", "Patient_24h_logCFU"]

#add baseline isolates when they're available
in_vivo_final = pd.merge(in_vivo_pivot, baseline_isos, on="Patient", how="left")

in_vivo_final = in_vivo_final[["Patient", "Strain", "Patient_0h_logCFU", "Patient_4h_logCFU", "Patient_24h_logCFU"]]
in_vivo_final.rename(columns = {"Strain" : "Baseline_Iso"}, inplace=True)
in_vivo_final.to_csv("Patient_logCFU_0424h.csv", index=False)

#### Finding all duplicate isolates

In [ ]:
#Now I'm concerned about overall duplicates

cnum_larger = pd.read_csv("cnum_subset.csv")

all_duplicates = cnum_larger.duplicated(subset=['subject_id', "specimen_collection_time_description", 'visit_raw'], keep=False)
cnum_larger[all_duplicates].to_csv("duplicate_isolates.csv", index=False)
all_patients = list(cnum_clinic.subject_id.unique())
baseline_patients = list(baseline_isos.Patient.unique())

the_escapee = list(set(all_patients) - set(baseline_patients))
print(the_escapee)

### Just get the CFU_mL for all days 

In [ ]:
cnum_day3on = cnum_clinic[(cnum_clinic["visit_raw"] == "V3/D3") | (cnum_clinic["visit_raw"] == "V4/D4") | (cnum_clinic["visit_raw"] == "V5/D5") | 
                                      (cnum_clinic["visit_raw"] == "V7/D10 TOC") | (cnum_clinic["visit_raw"] == "V8/D34 ET")]
cnum_day3on = cnum_day3on[cnum_day3on["specimen_collection_time_description"] != "4 hr Post Dose"]

day3on_duplicates = cnum_day3on.duplicated(subset=['subject_id', 'visit_raw'], keep=False)
#cnum_day3on[day3on_duplicates]

cnum_day3on_grouped = cnum_day3on.groupby(["subject_id", "visit_raw"])["CFU_ml"].max().reset_index()

cnum_day3on_pivot = cnum_day3on_grouped.pivot(
    columns = "visit_raw",
    index = "subject_id",
    values = "CFU_ml").reset_index()

cnum_day3on_pivot.iloc[:, 1:] = cnum_day3on_pivot.iloc[:, 1:].apply(np.log10)
cnum_day3on_pivot.columns=["Patient", "Day3_logCFU", "Day4_logCFU", "Day5_logCFU", "Day10_logCFU", "Day34_logCFU"]

#merge with timepoints from days 1 and 2
patient_full = pd.merge(in_vivo_final, cnum_day3on_pivot, on="Patient", how="left")

#Get some drops in there
patient_full["Patient_4hlogCFUdrop"] = patient_full.Patient_0h_logCFU - patient_full.Patient_4h_logCFU
columns_past_0h = ["Patient_4h_logCFU", "Patient_24h_logCFU", "Day3_logCFU", "Day4_logCFU", "Day5_logCFU", "Day10_logCFU", "Day34_logCFU"]
patient_full["Patient_maxCFUdrop_fromBaseline"] = patient_full.Patient_0h_logCFU - patient_full[columns_past_0h].min(axis=1)
all_timepoints = ["Patient_0h_logCFU", "Patient_4h_logCFU", "Patient_24h_logCFU", "Day3_logCFU", "Day4_logCFU", "Day5_logCFU", "Day10_logCFU", "Day34_logCFU"]
patient_full["Patient_maxCFUdrop_anyTime"] = patient_full[all_timepoints].max(axis=1) - patient_full[all_timepoints].min(axis=1)

patient_full['Baseline_Iso'] = patient_full['Baseline_Iso'].replace({"C000901": "C000902"})
patient_full.to_csv("Patient_logCFU_full.csv", index=False)

In [ ]:
#subbing in 0 CFU/mL points where indicated by the graph
patient_full_wZEROS = pd.read_csv(#redacted)
patient_full_wZEROS["Patient_4hlogCFUdrop"] = patient_full_wZEROS.Patient_0h_logCFU - patient_full_wZEROS.Patient_4h_logCFU
patient_full_wZEROS["Patient_maxCFUdrop"] = patient_full_wZEROS.Patient_0h_logCFU - patient_full_wZEROS[columns_past_0h].min(axis=1)
all_timepoints = ["Patient_0h_logCFU", "Patient_4h_logCFU", "Patient_24h_logCFU", "Day3_logCFU", "Day4_logCFU", "Day5_logCFU", "Day10_logCFU", "Day34_logCFU"]
all_timepoints_but_last = ["Patient_0h_logCFU", "Patient_4h_logCFU", "Patient_24h_logCFU", "Day3_logCFU", "Day4_logCFU", "Day5_logCFU", "Day10_logCFU"]
patient_full_wZEROS["Patient_maxCFUdrop_anyTime"] = patient_full_wZEROS[all_timepoints_but_last].max(axis=1) - patient_full_wZEROS[all_timepoints].min(axis=1)
patient_full_wZEROS.to_csv(#redacted, index=False)

### Adding TOC ("test of cure') CFU/mL at day 10 metric

In [ ]:
cnum_TOC = cnum_clinic[cnum_clinic["visit_raw"] == "V7/D10 TOC"]
cnum_TOC.to_csv("Clinic_TOC.csv", index=False)